In [1]:
from STT import STT_Engine
# import llm
# import resp_to_sound
import os
import sys
import socket
import time
import psutil
import netifaces as ni
import whisper

from pydub import AudioSegment
import simpleaudio as sa


In [2]:
# telemetry
UDP_PORT = 9998
IN_PORT = 9999
OUT_PORT = 10000

LOCAL_IP = ""
BROADCAST_IP1 = "255.255.255.255"
BROADCAST_IP2 = "192.168.0.255"

DISCOVERY_MESSAGE = b"WHERE_IS_MY_GIRLFRIEND"
EXPECTED_RESPONSE = b"IM_HERE"

TEXT_MESSAGE = b"TEXT"
AUDIO_MESSAGE = b"AUDIO"
TEXT_END_MESSAGE = b"TEXTEND"
AUDIO_END_MESSAGE = b"AUDIOEND"

END_MESSAGE = b"DONE"

MAX_TRY = 10
MAX_FILES = 10
INPUT_FOLDER = os.path.join(os.getcwd(),"input_files")
OUTPUT_FOLDER = os.path.join(os.getcwd(),"output_files")

# STT
mySTT = STT_Engine();
mySTT.setFilePath(os.path.join(INPUT_FOLDER,"input.wav"))

D:\_____ENGINEERING_____\Anaconda\exe\envs\lain-chat-pc\lib\site-packages\whisper\__init__.py
Model 'large-v2' found at D:\_____ENGINEERING_____\arduino\AnimationTFT-main\myConverter\pc_bot\models\large-v2.pt. Loading...


In [3]:
# def timestamp_to_int(filename):
#     t = filename[len("received_"):-len(".wav")]
#     return int(t.replace(":", "").replace("_", ""))

# def clean_old_files(folder):
#     all_files = os.listdir(folder)
#     all_files.sort(key=timestamp_to_int)

#     while len(all_files) > MAX_FILES:
#         to_be_removed = all_files.pop(0)
#         os.remove(os.path.join(folder, to_be_removed))

def receive_audio(local_ip,board_ip):
    filename = os.path.join(INPUT_FOLDER, f"input.wav")
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Deleted existing file: {filename}")
    
    print(f"received file to be saved as: {filename}")
        
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((local_ip, IN_PORT)) # pc bind its port
    s.settimeout(0.1)  
    s.listen()

    # with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s_ready:
    #     s_ready.connect((board_ip, OUT_PORT))
    #     s_ready.sendall(END_MESSAGE)
    #     s_ready.close()
            
    print(f'waiting for input from {board_ip}')

    # Continuous retry until a connection is established
    conn = None
    while not conn:
        try:
            conn, addr = s.accept()
            if addr[0] != board_ip:
                conn.close()  # Close if the IP doesn't match
                conn = None
        except socket.timeout:
            continue  # Retry on timeout
        
    with conn:
        print(f'{addr} connected')
        if addr[0] == board_ip:
            with open(filename, "wb") as f:
                while True:
                    chunk = conn.recv(3072)
                    if END_MESSAGE in chunk:
                        f.write(chunk.split(END_MESSAGE)[0])
                        break
                    f.write(chunk)
            print(f'saved received audio: {filename}\n')
    
                # optional; limit num of audio files in folder
                # clean_old_files(INPUT_FOLDER)

def send_MSG(MSG, sock, board_ip):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((board_ip, OUT_PORT))
        s.sendall(MSG)
        print("MSG", MSG, " sent.\n")
        time.sleep(1)
    
def send_audio(file_name, sock, board_ip):
    final_path = os.path.join(OUTPUT_FOLDER,file_name)
    if not os.path.exists(final_path):
        print(f"Error: File '{final_path}' does not exist.")
        return
    print(f"File: {final_path}")

    send_MSG(AUDIO_MESSAGE,board_ip)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((board_ip, OUT_PORT))

        with open(final_path, "rb") as f:
            while chunk := f.read(3072):
                s.sendall(chunk)

        # send a packet signaling NEXT
        time.sleep(1)
        s.sendall(AUDIO_END_MESSAGE)
        print("Audio file sent.\n")

def send_text(text, sock, board_ip):
    send_MSG(TEXT_MESSAGE,board_ip)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((board_ip, OUT_PORT))

        s.sendall(text.encode('utf-8'))
    
        # send a packet signaling NEXT
        time.sleep(1)
        s.sendall(TEXT_END_MESSAGE)
        print("Text file sent.\n")

def get_correct_ip():
    for iface, addrs in psutil.net_if_addrs().items():
        if "Wi-Fi" in iface or "Wireless" in iface:  # Ensure it's the Wi-Fi adapter
            for addr in addrs:
                if addr.family == 2 and addr.address.startswith("192.168.0."):  # family = 2 --> AF_INET = IPv4
                    return addr.address

def find_board():
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
        sock.settimeout(1)

        # local/PC
        local_ip = get_correct_ip()  # Get the IP address associated with the hostname
        print("local IP is: " + local_ip)
        sock.bind((local_ip, UDP_PORT))
        print("sock name after bind: "+str(sock.getsockname()))

        board_addr = None

        for _ in range(MAX_TRY):
            sock.sendto(DISCOVERY_MESSAGE, (BROADCAST_IP1, UDP_PORT))
            # sock.bind(("192.168.0.27", UDP_PORT))
            try:
                for _ in range(5):
                    data, addr = sock.recvfrom(1024) # addr = (ip, port)
                    if data.strip() == EXPECTED_RESPONSE and addr[1] == UDP_PORT:
                        board_addr = addr
                        print(f"Receiving response: {data.strip()}")
                        print(f"board addr: {board_addr}\n")
                        # terminate UDP channel
                        time.sleep(2)
                        print("sending confirmation.")
                        sock.sendto(END_MESSAGE, board_addr)
                        return board_addr,local_ip
            except socket.timeout:
                print("timeout, retry...")
            time.sleep(0.5) # have to, otherwise keep receiving self-msg before board msg parsed here

    if not board_addr:
        print("can't find board!\n")
        return None,None

def play_audio():
    audio_file = os.path.join(INPUT_FOLDER, "input.wav")
    print("Playing received audio...")
    try:
        sound = AudioSegment.from_wav(audio_file)
        play_obj = sa.play_buffer(
            sound.raw_data,
            num_channels=sound.channels,
            bytes_per_sample=sound.sample_width,
            sample_rate=sound.frame_rate
        )
        play_obj.wait_done()
        
    except Exception as e:
        print(f"Error playing audio: {e}")

# Main

In [4]:
os.makedirs("input_files", exist_ok=True)
os.makedirs("output_files", exist_ok=True)

# chat_history = llm.ChatHistory(max_size=10)

# connect board
board_addr, LOCAL_IP = find_board()

local IP is: 192.168.0.27
sock name after bind: ('192.168.0.27', 9998)
timeout, retry...
timeout, retry...
Receiving response: b'IM_HERE'
board addr: ('192.168.0.18', 9998)

sending confirmation.


In [5]:
while True:
    # receive audio
    receive_audio(LOCAL_IP,board_addr[0])
    # inference and play audio
    play_audio()
    result = mySTT.STT_inference()
    print("Inference result: ", result)
    if result:
        send_text(result,board_addr[0])
    # send audio
    # send_audio("hellonavi_16.wav",board_addr[0])
    
    send_MSG(END_MESSAGE,board_addr[0])
    time.sleep(1)

# --------------------------------------------------------
# while(True):
#     # get audio from board
#     input_file = get_audio(board_addr)
    
#     # audio -> text
#     text = audio_to_text.audio_to_text(input_file)  
    
#     # ai gen response
#     resp = llm.get_ai_response(text, chat_history)
    
#     # resp -> voice
#     output_path = resp_to_sound.resp_to_sound(resp)
    
#     # send back audio
#     send_audio(output_path, board_addr[0])
    
#     print(f'sent {output_path} to {board_addr}:10000')

Deleted existing file: D:\_____ENGINEERING_____\arduino\AnimationTFT-main\myConverter\pc_bot\input_files\input.wav
received file to be saved as: D:\_____ENGINEERING_____\arduino\AnimationTFT-main\myConverter\pc_bot\input_files\input.wav
waiting for input from 192.168.0.18
('192.168.0.18', 59852) connected
saved received audio: D:\_____ENGINEERING_____\arduino\AnimationTFT-main\myConverter\pc_bot\input_files\input.wav

Playing received audio...
Inference result:  User: 为忠


KeyboardInterrupt: 